In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU

def unet(input_shape, num_classes):
    inputs = Input(input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

    output_layer = Conv2D(num_classes, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=output_layer)

    return model

# Define input shape, number of classes, and batch size
input_shape = (256, 256, 1)
num_classes = 1
batch_size = 16

# Load dataset
train_ds = (...path_to_dataset)
val_ds = (...path_to_dataset)
test_ds = (...path_to_dataset)

# Preprocess dataset
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(input_shape[0], input_shape[1]),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])

train_ds = train_ds.map(lambda x, y: (resize_and_rescale(x, training=True), y))
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (resize_and_rescale(x, training=False), y))
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

test
# Initialize the U-Net model
model = unet(input_shape, num_classes)

# Compile the model
model.compile(optimizer=Adam(),
              loss=BinaryCrossentropy(),
              metrics=['accuracy', MeanIoU(num_classes=num_classes)])

# Train the model
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=50,  # Modify this based on your requirement
                    batch_size=batch_size)

# Evaluate the model on the test dataset
test_loss, test_accuracy, test_mean_iou = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Mean IoU: {test_mean_iou}")

# Optionally, save the model
model.save('path/to/save/model')


In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import os

def load_and_preprocess_image(image_path, input_shape):
    # Load image
    image = Image.open(image_path)
    image = image.resize(input_shape[:2])  # Assuming input_shape is like (height, width, channels)
    image = np.array(image)
    image = image / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Load the trained model
model = load_model('path/to/saved/model')

# Load and preprocess new images
# Example path to a new image
new_image_path = 'path/to/new/image.jpg'
input_shape = (256, 256, 1)  # Same as used during training
preprocessed_image = load_and_preprocess_image(new_image_path, input_shape)

# Predict tissue mask
predicted_mask = model.predict(preprocessed_image)


In [ ]:
def save_mask(mask, save_path, image_name):
    # Convert the mask to an image
    mask_image = Image.fromarray((mask[0, :, :, 0] * 255).astype('uint8'))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    mask_image.save(os.path.join(save_path, image_name))

# Example save path
save_path = './predicted_tissue_masks'
image_name = 'predicted_mask.jpg'

# Save the predicted mask
save_mask(predicted_mask, save_path, image_name)
